In [3]:

import os
# os.environ['CUDA_VISIBLE_DEVICES'] = ''
# os.environ['TF_KERAS'] = '1'

import tensorflow as tf
# import keras_bert
from functools import partial
import importlib
import data_loader
importlib.reload(data_loader)
print(tf.__version__)
DataLoader = data_loader.DataLoader

# keras_bert.backend.TF_KERAS = 1

config = './config/hub2-1.json'
# print(keras_bert.backend.TF_KERAS)

2.0.0


In [4]:
tf.test.is_gpu_available()

True

In [5]:
dl = DataLoader(config=config, data='../data', n_max=None)
config_file = os.path.join(dl.data, 'bert_model', 'bert_config.json')
checkpoint_file = 

INFO:root:DataLoader initializing
INFO:root:reading lab data from ../data/soundAttGAN & ../data/soundAttGAN/koreancorpus.xlsx
INFO:root:total number of data: 1925
INFO:root:reading hub data from ../data/KsponSpeech_01
DEBUG:root:../data/KsponSpeech_01/KsponSpeech_0094
INFO:root:total number of data: 709
INFO:root:Reading vocab from ../data/bert_model/vocab.txt
INFO:root:The number of vocab is 119547
INFO:root:Build done
INFO:root:{'batch_size': 256,
 'epochs': 10000,
 'fmax': 8000,
 'hop_length': 250,
 'kernel_size': 4,
 'learning_rate': 0.001,
 'max_len': 32,
 'max_sec': 2,
 'n_fft': 510,
 'n_max': 100000,
 'n_mels': 128,
 'output_size': 256,
 'sr_hub': 16000,
 'sr_lab': 22050,
 'top_db': 80.0,
 'win_length': 510,
 'window': 'hann'}


In [6]:
it = iter(dl.train_generator(data='hub', return_text=True))

In [7]:
next(it)

[101,
 8924,
 31401,
 25503,
 118671,
 9682,
 9539,
 18784,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [8]:
def _get_embedding_table():
    ckpt_loader = tf.train.load_checkpoint(checkpoint_file)
#     model = keras_bert.load_trained_model_from_checkpoint(config_file=config_file,
#                                                          checkpoint_file=checkpoint_file,
#                                                          training=False,
#                                                          trainable=None,
#                                                          output_layer_num=1,
#                                                          seq_len=dl.max_len)
#     embed_table = keras_bert.get_token_embedding(model)
#     del(model)
    embed_table = ckpt_loader.get_tensor('bert/embeddings/word_embeddings')
    del(ckpt_loader)
    return embed_table

In [70]:
import tensorflow as tf
from utils import *

keras = tf.keras

BatchNorm = keras.layers.BatchNormalization
Conv2D = keras.layers.Conv2D
DConv = keras.layers.Conv2DTranspose
Dense = keras.layers.Dense


class AutoEncoder(keras.models.Model):
    def __init__(self, config, input_shape, **kwargs):
        super(AutoEncoder, self).__init__(**kwargs)
        self.config = load_config(config)
        self.audio_shape = input_shape

        self.act_fn = keras.layers.LeakyReLU()
        self.kernel_size = self.config['kernel_size']
        
        self.embedding_table = _get_embedding_table()

        self.conv = [
            Conv2D(filters=32, kernel_size=self.kernel_size, strides=(2, 2), padding='same', activation=None,
                   input_shape=input_shape),
            BatchNorm(),
            self.act_fn,
            Conv2D(filters=64, kernel_size=self.kernel_size, strides=(2, 2), padding='same', activation=None),
            BatchNorm(),
            self.act_fn,
            Conv2D(filters=128, kernel_size=self.kernel_size, strides=(2, 2), padding='same', activation=None),
            BatchNorm(),
            self.act_fn,
            Conv2D(filters=256, kernel_size=self.kernel_size, strides=(2, 2), padding='same', activation=None),
            BatchNorm(),
            self.act_fn,
            Conv2D(filters=512, kernel_size=self.kernel_size, strides=(2, 2), padding='same', activation=None),
            BatchNorm(),
            self.act_fn,
            #             keras.layers.Flatten(),
            #             Dense(units=output_size, activation=None),
            #             self.act_fn
        ]

        self.dconv = [
            # 16
            DConv(filters=512, kernel_size=self.kernel_size, strides=(2, 2), padding='same', activation=None),
            BatchNorm(),
            self.act_fn,
            # 32
            DConv(filters=256, kernel_size=self.kernel_size, strides=(2, 2), padding='same', activation=None),
            BatchNorm(),
            self.act_fn,
            # 64
            DConv(filters=128, kernel_size=self.kernel_size, strides=(2, 2), padding='same', activation=None),
            BatchNorm(),
            self.act_fn,

            DConv(filters=64, kernel_size=self.kernel_size, strides=(2, 2), padding='same', activation=None),
            BatchNorm(),
            self.act_fn,

            DConv(filters=1, kernel_size=self.kernel_size, strides=(2, 2), padding='same', activation=None),
            keras.layers.ReLU(max_value=80.0)
        ]
        self.embed_hidden = Dense(units=self.hidden_size[0] * self.hidden_size[1])

    @tf.function
    def call(self, inputs, **kwargs):
        audio, text = inputs
        audio = tf.expand_dims(audio, -1)

        for layer in self.conv:
            audio = layer(audio)
            
        audio = self._concat_text(audio, text)
        for layer in self.dconv:
            audio = layer(audio)
        audio = tf.squeeze(audio, axis=-1)
        
        return audio
    
    @property
    def hidden_size(self):
        return int(self.audio_shape[0] / 2**5), int(self.audio_shape[1] / 2**5)
    
    def _concat_text(self, audio, text):
        text = tf.gather(self.embedding_table, text)
        text = self.embed_hidden(text)
        text = tf.reshape(text, shape=[-1, 32, *self.hidden_size])
        text = tf.transpose(text, perm=[0, 2, 3, 1])
        audio = tf.concat([audio, text], axis=-1)
        return audio
        
        
        


In [71]:
# del(autoencoder)
autoencoder = AutoEncoder(config=config, input_shape=dl.stft_shape)

In [72]:
dataset_audio = tf.data.Dataset.from_generator(partial(dl.train_generator, data='hub'), output_types=tf.float32)
dataset_text = tf.data.Dataset.from_generator(partial(dl.train_generator, data='hub', return_text=True), output_types=tf.int32)

dataset = tf.data.Dataset.zip((dataset_audio, dataset_text))
dataset = dataset.shuffle(buffer_size=dl.batch_size * 10).batch(16)

inputs = next(iter(dataset))

In [73]:
autoencoder(inputs)

<tf.Tensor: id=15941, shape=(16, 256, 128), dtype=float32, numpy=
array([[[1.70086582e-06, 0.00000000e+00, 0.00000000e+00, ...,
         1.11190911e-05, 0.00000000e+00, 0.00000000e+00],
        [1.07899396e-05, 0.00000000e+00, 1.63386198e-04, ...,
         0.00000000e+00, 2.88854862e-05, 0.00000000e+00],
        [6.64469917e-05, 9.95200753e-05, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         7.20243906e-06, 3.92513248e-05, 0.00000000e+00],
        [1.98937778e-05, 3.19812261e-06, 0.00000000e+00, ...,
         2.69277643e-06, 0.00000000e+00, 1.67227026e-05],
        [0.00000000e+00, 9.46991076e-06, 8.35503761e-06, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[2.65257731e-05, 0.00000000e+00, 0.00000000e+00, ...,
         3.97414115e-05, 0.00000000e+00, 0.00000000e+00],
        [2.48286597e-06, 0.00000000e+00, 3.98206234e-04, ...,
         0.000000

In [46]:
for key, value in ckpt_loader.get_variable_to_shape_map().items():
    if 'embeddings' in key:
        print(key, ':', value)
    # bert/embeddings/word_embeddings 사용

NameError: name 'ckpt_loader' is not defined